In [3]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [4]:
!wget https://github.com/vncorenlp/VnCoreNLP/archive/refs/heads/master.zip
!unzip master.zip
!rm -f master.zip
!mv VnCoreNLP-master VnCoreNLP
!pip install py_vncorenlp --quiet

--2024-07-10 15:27:44--  https://github.com/vncorenlp/VnCoreNLP/archive/refs/heads/master.zip
Resolving github.com (github.com)... 140.82.116.4
Connecting to github.com (github.com)|140.82.116.4|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://codeload.github.com/vncorenlp/VnCoreNLP/zip/refs/heads/master [following]
--2024-07-10 15:27:44--  https://codeload.github.com/vncorenlp/VnCoreNLP/zip/refs/heads/master
Resolving codeload.github.com (codeload.github.com)... 140.82.116.9
Connecting to codeload.github.com (codeload.github.com)|140.82.116.9|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: unspecified [application/zip]
Saving to: ‘master.zip’

master.zip              [                 <=>] 160.84M  23.8MB/s    in 6.8s    

2024-07-10 15:27:51 (23.8 MB/s) - ‘master.zip’ saved [168648997]

Archive:  master.zip
62bbc58fe5d113c898eae112656be97dcf50b3a0
   creating: VnCoreNLP-master/
  inflating: VnCoreNLP-master/LICENSE.md  


In [5]:
from string import punctuation
import py_vncorenlp
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, classification_report
from sklearn.preprocessing import LabelEncoder
import numpy as np
from sklearn.linear_model import LogisticRegression

In [6]:
def clean_text(input_str):
  my_punctuation = punctuation.replace('_', '')
  output_str = vncorenlp_model.word_segment(input_str)
  output_str = ''.join(output_str)
  output_str = output_str.translate(str.maketrans('','',my_punctuation))
  output_str = ' '.join(output_str.split())
  return output_str.lower()

In [7]:
vncorenlp_model = py_vncorenlp.VnCoreNLP(
    annotators=["wseg"],
    save_dir='/content/VnCoreNLP'
)

In [9]:
# Load the TSV file into a pandas DataFrame
train_df = pd.read_csv("/content/drive/MyDrive/XLNNTN/TH/08/train.tsv", sep='\t',header=None, names=['text','label'])
valid_df = pd.read_csv("/content/drive/MyDrive/XLNNTN/TH/08/val.tsv", sep='\t',header=None, names=['text','label'])
test_df = pd.read_csv("/content/drive/MyDrive/XLNNTN/TH/08/test.tsv", sep='\t',header=None, names=['text','label'])

train_df['clean_text'] = train_df['text'].apply(clean_text)
test_df['clean_text'] = test_df['text'].apply(clean_text)

In [10]:
train_df.head()

,text,label,clean_text
0,Ba Lan sa thải HLV Santos sau sáu trận. Liên đ...,thethao,ba_lan sa_thải hlv santos sau sáu trận liên_đo...
1,Trữ trứng để sinh con được bao lâu?. Tôi 32 tu...,suckhoe,trữ trứng để sinh con được bao_lâu tôi 32 tuổi...
2,Thường xuyên đau lưng là bệnh gì?. Lưng vợ tôi...,suckhoe,thường_xuyên đau lưng là bệnh gì lưng vợ tôi t...
3,Người mẹ bỏ chữa ung thư để tìm sự sống cho co...,doisong,người mẹ bỏ chữa ung_thư để tìm sự sống cho co...
4,"Nguy cơ Covid đồng nhiễm virus, vi khuẩn gây b...",suckhoe,nguy_cơ covid đồng nhiễm virus vi_khuẩn gây bệ...


In [11]:
print(train_df.shape)
print(test_df.shape)

(31745, 3)
(9115, 3)


In [12]:
train_text = train_df['clean_text'].tolist()
train_label = train_df['label'].tolist()

test_text = test_df['clean_text'].tolist()
test_label = test_df['label'].tolist()

corpus = train_text + test_text
vectorizer = TfidfVectorizer()
vectorizer.fit(corpus)

TfidfVectorizer()

In [13]:
le = LabelEncoder()
le.fit(train_label)

LabelEncoder()

In [14]:
X_train = vectorizer.transform(train_text)
y_train = le.transform(train_label)

X_test = vectorizer.transform(test_text)
y_test = le.transform(test_label)

print(X_train.shape)
print(len(y_train))
print(X_test.shape)
print(len(y_test))

(31745, 33979)
31745
(9115, 33979)
9115


In [15]:
clf = LogisticRegression(random_state=0,solver='lbfgs', max_iter=200, C=2).fit(X_train, y_train)
y_pred = clf.predict(X_test)

In [16]:
print(classification_report(y_test, y_pred, digits=4, target_names=le.classes_))

              precision    recall  f1-score   support

     doisong     0.7706    0.7104    0.7393       473
     giaoduc     0.9236    0.8917    0.9073       637
     khoahoc     0.8787    0.8403    0.8591       595
      kinhte     0.8809    0.8870    0.8839       867
     suckhoe     0.9419    0.9734    0.9574      1917
     thegioi     0.9421    0.9344    0.9382      1966
     thethao     0.9870    0.9828    0.9849      1626
      thoisu     0.8675    0.8994    0.8832      1034

    accuracy                         0.9220      9115
   macro avg     0.8990    0.8899    0.8942      9115
weighted avg     0.9215    0.9220    0.9215      9115



## BERT

In [23]:
train_df = pd.read_csv("/content/drive/MyDrive/XLNNTN/TH/08/train.tsv", sep='\t',header=None, names=['text','label'])
val_df = pd.read_csv("/content/drive/MyDrive/XLNNTN/TH/08/val.tsv", sep='\t',header=None, names=['text','label'])
test_df = pd.read_csv("/content/drive/MyDrive/XLNNTN/TH/08/test.tsv", sep='\t',header=None, names=['text','label'])

In [24]:
from sklearn.preprocessing import LabelEncoder

# Khởi tạo LabelEncoder và chuyển đổi nhãn
label_encoder = LabelEncoder()
train_labels = label_encoder.fit_transform(train_df['label'])
val_labels = label_encoder.transform(val_df['label'])
test_labels = label_encoder.transform(test_df['label'])


In [25]:
from transformers import BertTokenizer

# Sử dụng tokenizer của BERT
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

def tokenize_data(data, text_column='text', max_length=128):
    return tokenizer(
        data[text_column].tolist(),
        padding=True,
        truncation=True,
        max_length=max_length,
        return_tensors="pt"
    )

# Tokenize dữ liệu huấn luyện, validation, và kiểm tra với tên cột đúng
train_encodings = tokenize_data(train_df, text_column='text')
val_encodings = tokenize_data(val_df, text_column='text')
test_encodings = tokenize_data(test_df, text_column='text')


In [26]:
import torch
from torch.utils.data import DataLoader, TensorDataset

# Tạo dataset cho DataLoader
train_dataset = TensorDataset(
    train_encodings['input_ids'],
    train_encodings['attention_mask'],
    torch.tensor(train_labels)
)
val_dataset = TensorDataset(
    val_encodings['input_ids'],
    val_encodings['attention_mask'],
    torch.tensor(val_labels)
)

train_loader = DataLoader(train_dataset, batch_size=16, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=16, shuffle=False)


In [28]:
from transformers import BertForSequenceClassification, AdamW

# Số lượng nhãn
num_labels = len(label_encoder.classes_)

# mô hình BERT cho phân loại
model = BertForSequenceClassification.from_pretrained('bert-base-uncased', num_labels=num_labels)

device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')
model.to(device)

# optimizer
optimizer = AdamW(model.parameters(), lr=5e-5)

# Huấn luyện mô hình
for epoch in range(4):
    model.train()
    total_loss = 0
    for batch in train_loader:
        optimizer.zero_grad()
        input_ids, attention_mask, labels = [x.to(device) for x in batch]
        outputs = model(input_ids, attention_mask=attention_mask, labels=labels)
        loss = outputs.loss
        loss.backward()
        optimizer.step()
        total_loss += loss.item()
    avg_train_loss = total_loss / len(train_loader)

    # Đánh giá trên validation set
    model.eval()
    total_eval_loss = 0
    with torch.no_grad():
        for batch in val_loader:
            input_ids, attention_mask, labels = [x.to(device) for x in batch]
            outputs = model(input_ids, attention_mask=attention_mask, labels=labels)
            loss = outputs.loss
            total_eval_loss += loss.item()
    avg_val_loss = total_eval_loss / len(val_loader)

    print(f'Epoch {epoch+1} - Training Loss: {avg_train_loss} - Validation Loss: {avg_val_loss}')


Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Epoch 1 - Training Loss: 0.7270962964543974 - Validation Loss: 0.4804081721794775
Epoch 2 - Training Loss: 0.36191258528539605 - Validation Loss: 0.3779588126514465
Epoch 3 - Training Loss: 0.24383315466143782 - Validation Loss: 0.3654601074145358
Epoch 4 - Training Loss: 0.1776474112305809 - Validation Loss: 0.3764150499310709


In [29]:
# Đánh giá mô hình trên tập test
test_dataset = TensorDataset(
    test_encodings['input_ids'],
    test_encodings['attention_mask'],
    torch.tensor(test_labels)
)
test_loader = DataLoader(test_dataset, batch_size=16, shuffle=False)

model.eval()
predictions, true_labels = [], []

with torch.no_grad():
    for batch in test_loader:
        input_ids, attention_mask, labels = [x.to(device) for x in batch]
        outputs = model(input_ids, attention_mask=attention_mask)
        logits = outputs.logits
        predictions.extend(torch.argmax(logits, dim=1).cpu().numpy())
        true_labels.extend(labels.cpu().numpy())

report = classification_report(true_labels, predictions, target_names=label_encoder.classes_)
print(report)


              precision    recall  f1-score   support

     doisong       0.74      0.60      0.66       473
     giaoduc       0.89      0.91      0.90       637
     khoahoc       0.82      0.76      0.79       595
      kinhte       0.79      0.88      0.83       867
     suckhoe       0.95      0.93      0.94      1917
     thegioi       0.93      0.93      0.93      1966
     thethao       0.99      0.97      0.98      1626
      thoisu       0.78      0.85      0.82      1034

    accuracy                           0.89      9115
   macro avg       0.86      0.85      0.86      9115
weighted avg       0.89      0.89      0.89      9115



[tham khảo](https://www.kaggle.com/code/tanulsingh077/deep-learning-for-nlp-zero-to-transformers-bert?scriptVersionId=31944644&cellId=64)